# Scanwich Group 2 - ECE4510  
### Polly Jane Bates, Sivasakthi Muthukumar, Sam Walker

<p>Press "space" to note what objects were detected.<br>
Press "g" to clear the objects list.<br>
Press "q" to exit the program.</p>

Scan the arUco marker to see the nutrition facts of the foods detected. <br>
You may need to update openCV to a different version to run the ArUco functions.

In [72]:
import cv2
import numpy as np
from ultralytics import YOLO

import logging
logging.getLogger("ultralytics").setLevel(logging.ERROR)  # supress frame detection outputs


# Load each model
model1 = YOLO('bestEgg.pt')
model2 = YOLO('bestCereal.pt')
model3 = YOLO('bestBananas.pt')
model4 = YOLO('bestApples.pt')
model5 = YOLO('bestPotatoes.pt')
model6 = YOLO('bestBread.pt') 

# Create a blank image as a canvas
im_src = np.zeros((1050, 780, 3), np.uint8)

# Load in scanwich header and nutrition facts label
scanwich = cv2.imread("SCANWICH.jpg")
nutrition_label = cv2.imread("NutritionFactsCropped.jpg")

# Get the dimensions of canvas and images
height, width = im_src.shape[:2]
height_scanwich, width_scanwich = scanwich.shape[:2]
height_nut, width_nut = nutrition_label.shape[:2]

# Calculate the scaling factor to fit header image and nutrition facts label
# in the canvas maintaining aspect ratios
scale = min(width / width_scanwich, height / height_scanwich)
scale_nut = min(width / width_nut, height / height_nut)

new_w = int(width_scanwich * scale)
new_h = int(height_scanwich * scale)

new_w_nut = int(width_nut * scale_nut)
new_h_nut = int(height_nut * scale_nut)

# Resize the header image and nutrition facts label
resized_scanwich = cv2.resize(scanwich, (new_w, new_h))
resized_nut = cv2.resize(nutrition_label, (new_w_nut, new_h_nut))

# Place the resized scanwich label into the canvas image
im_src[0: new_h, 0: new_w] = resized_scanwich
# Place the resized nutrition facts label into the canvas
im_src[150: new_h_nut+150, 0:new_w_nut] = resized_nut

# Locations on the canvas to place the nutrition fact values
calories_loc = (220,380)
fat_total_loc = (230, 510)
fat_sat_loc = (380, 570)
fat_tran_loc = (330,630)
cholesterol_loc = (300,690)
sodium_loc = (210,750)
carb_total_loc = (410,800)
fiber_loc = (370,860)
sugars_loc = (290,920)
protein_loc = (210, 980)

# settings for the label
color = (255, 0, 255)
fontScale = 3
thickness = 2
font = cv2.FONT_HERSHEY_PLAIN

# corners of the canvas
pts_src = np.empty((0,2),dtype=np.int32)
pts_src = np.append(pts_src, [(0,0)], axis=0)
pts_src = np.append(pts_src, [(width-1,0)], axis=0)
pts_src = np.append(pts_src, [(width-1,height-1)], axis=0)
pts_src = np.append(pts_src, [(0,height-1)], axis=0)



# Save the canvas with no data 
blank = im_src.copy()

# Define ArUco marker dictionary
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_250)

# Open the webcam
cap = cv2.VideoCapture(1)  # 0 for the default webcam
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

detected_objects = []  # List of detected object labels for the current frame
frozen_objects = []    # Persistent list of objects to display

foods =  ["Apple", "bananas", "Bread", "cereal", "egg", "potato"]
cals =   [95,  104,  110, 190, 74,   168]  
t_fat =  [0.3, 0.39, 2,   1,   4.97, 0.2]  #g
s_fat =  [0.1, 0.13, 0,   0,   1.55, 0.1]  #g
tr_fat = [0,   0,    0,   0,   0,    0]  #g
chol =   [0,   0,    0,   0,   212,  0] #mg
sodi =   [2,   1,    170, 200, 70,   25] #mg
t_carb = [25,  27,   21,  47,  0.38, 37]  #g
fib   =  [4.4, 3.1,  3,   7,   0,    4]  #g
sug   =  [ 19, 14.4, 3,   17,  0.38, 1.9]  #g
pro   =  [0.5, 1.3,  4,   5,   6.29, 4.5] #g

calories = 0
fat_total = 0
fat_sat = 0
fat_tran = 0
cholesterol = 0
sodium = 0
carb_total = 0
fiber = 0
sugars = 0
protein = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform object detection
    # results = model(frame, stream=True)

    # Object detection on each model
    results1 = model1(frame, stream = True)
    results2 = model2(frame, stream = True)
    results3 = model3(frame, stream = True)
    results4 = model4(frame, stream = True)
    results5 = model5(frame, stream = True)
    results6 = model6(frame, stream = True)


    # Reset the nutrition totals at the start of each frame
    calories = 0
    fat_total = 0
    fat_sat = 0
    fat_tran = 0
    cholesterol = 0
    sodium = 0
    carb_total = 0
    fiber = 0
    sugars = 0
    protein = 0

    detected_objects.clear()
    
    # EGGS MODEL # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # Confidence set to 90%
    
    for result in results1:
        boxes = result.boxes.xyxy  # Bounding boxes
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls  
        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if conf >= 0.9:  
                x1, y1, x2, y2 = map(int, box)  # Convert to int for OpenCV
                label = model1.names[int(cls_id)]  # Just the class name
                detected_objects.append(label)   # Add only the label
                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    
    # CEREAL MODEL # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    # Confidence set to 80%
    
    for result in results2:
        boxes = result.boxes.xyxy  # Bounding boxes
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls  
        
        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if conf >= 0.8: 
                
                x1, y1, x2, y2 = map(int, box)  # Convert to int for OpenCV
                label = model2.names[int(cls_id)]  # Just the class name
                detected_objects.append(label)   # Add only the label
                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                
    # BANANAS MODEL # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    # Confidence set to 70%
    
    for result in results3:
        boxes = result.boxes.xyxy  # Bounding boxes
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls 
        
        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if conf >= 0.7: 

                x1, y1, x2, y2 = map(int, box)  # Convert to int for OpenCV
                label = model3.names[int(cls_id)]  # Just the class name
                detected_objects.append(label)   # Add only the label
                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


    # APPLES MODEL # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    # Confidence set to 90%
    
    for result in results4:
        boxes = result.boxes.xyxy  # Bounding boxes
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls 
        
        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if conf >= 0.90: 

                x1, y1, x2, y2 = map(int, box)  # Convert to int for OpenCV
                label = model4.names[int(cls_id)]  # Just the class name
                detected_objects.append(label)   # Add only the label
                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


    # POTATOES MODEL # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    # Confidence set to 50%
    
    for result in results5:
        boxes = result.boxes.xyxy  # Bounding boxes
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls 
        
        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if conf >= 0.50: 

                x1, y1, x2, y2 = map(int, box)  # Convert to int for OpenCV
                label = model5.names[int(cls_id)]  # Just the class name
                detected_objects.append(label)   # Add only the label
                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


    # BREAD MODEL # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    # Confidence set to 90%
    
    for result in results6:
        boxes = result.boxes.xyxy  # Bounding boxes
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls 
        
        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if conf >= 0.90: 

                x1, y1, x2, y2 = map(int, box)  # Convert to int for OpenCV
                label = model6.names[int(cls_id)]  # Just the class name
                detected_objects.append(label)   # Add only the label
                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Detect ArUco markers
    corners, ids, _ = cv2.aruco.detectMarkers(frame, dictionary)

    if ids is not None:
        
        # Get the corners of the marker
        pts_dst = corners[0][0]   
        # Calculate the center of the marker
        center = np.mean(pts_dst, axis=0)
        # Scale factor to expand the marker's bounding box 
        scale_factor = 4 
        # Extend the corners outward
        pts_dst = (pts_dst - center) * scale_factor + center

        # Sum up the nutrition values for frozen objects
        for i in range(len(foods)): 
            if foods[i] in frozen_objects:
                calories    += cals[i]
                fat_total   += t_fat[i] 
                fat_sat     += s_fat[i]
                fat_tran    += tr_fat[i]
                cholesterol += chol[i]
                sodium      += sodi[i]
                carb_total  += t_carb[i]
                fiber       += fib[i]
                sugars      += sug[i]
                protein     += pro[i]
                print(foods[i])

        # Round the values to 2 decimal places
        calories = round(calories, 2)
        fat_total = round(fat_total, 2)
        fat_sat = round(fat_sat, 2)
        fat_tran = round(fat_tran, 2)
        cholesterol = round(cholesterol, 2)
        sodium = round(sodium, 2)
        carb_total = round(carb_total, 2)
        fiber = round(fiber, 2)
        sugars = round(sugars, 2)
        protein = round(protein, 2)

        # Place text on the canvas
        im_src = blank.copy()  # Reset the canvas to the original blank state
        # Add the text 
        im_src = cv2.putText(im_src, str(calories), calories_loc, font,            fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(fat_total)+'g', fat_total_loc, font,      fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(fat_sat)+'g', fat_sat_loc, font,          fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(fat_tran)+'g', fat_tran_loc, font,        fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(cholesterol)+'mg', cholesterol_loc, font, fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(sodium)+'mg', sodium_loc, font,           fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(carb_total)+'g', carb_total_loc, font,    fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(fiber)+'g', fiber_loc, font,              fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(sugars)+'g', sugars_loc, font,            fontScale, color, thickness, cv2.LINE_AA)
        im_src = cv2.putText(im_src, str(protein)+'g', protein_loc, font,          fontScale, color, thickness, cv2.LINE_AA)

        # Place the canvas on the AR marker
        tform, status = cv2.findHomography(pts_src, pts_dst)
        
        # Warp the updated canvas onto the marker
        im_temp = cv2.warpPerspective(im_src, tform, (frame.shape[1], frame.shape[0]))

        # Mask the area of the marker in the frame
        cv2.fillConvexPoly(frame, pts_dst.astype(int), (0, 0, 0), cv2.LINE_AA)

        # Add the warped canvas onto the frame
        frame = cv2.add(frame, im_temp)

        # list foods in upper left corner on screen
        y_offset = 50  
        for food in frozen_objects:
            if food in foods:  # Check if the detected food is in the foods list
                cv2.putText(frame, food, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                y_offset += 40  # Move down for the next item

    
    # Display the frame
    cv2.imshow('frame', frame)

    # key press events
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Quit 
        break
    elif key == ord(' '):  # Add current objects to frozen list
        frozen_objects.extend(detected_objects)
        frozen_objects = list(set(frozen_objects))  # Ensure the list has unique items
        print(frozen_objects)
    elif key == ord('g'):  # Clear the frozen list
        frozen_objects.clear()


# clean up
cap.release()
cv2.destroyAllWindows() 

['potato', 'egg', 'Apple']
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Apple
egg
potato
Appl